In [5]:
import pandas as pd

# Load all data files (if separate)
files = ['./data/2004.csv', './data/2005.csv', './data/2006.csv','./data/2007.csv', './data/2008.csv', './data/2009.csv','./data/2010.csv', './data/2011.csv', './data/2012.csv','./data/2013.csv', './data/2014.csv', './data/2015.csv','./data/2016.csv', './data/2017.csv', './data/2018.csv','./data/2019.csv', './data/2020.csv', './data/2021.csv','./data/2022.csv', './data/2023.csv', './data/2024.csv']  
data = pd.concat([pd.read_csv(file) for file in files])

# Ensure the data is sorted by date and contract
data = data.sort_values(by=['Date', 'Price'])


In [6]:
data

,Date,Price
0,2003-05-15,-32.50
1,2003-05-16,-33.00
2,2003-05-17,-33.00
3,2003-05-18,-33.00
4,2003-05-19,-36.50
...,...,...
479,2024-07-05,44.00
480,2024-07-08,55.00
481,2024-07-09,56.25
482,2024-07-10,52.75


In [7]:
# Convert Date column to datetime format
data['Date'] = pd.to_datetime(data['Date'])

# Sort by date
data = data.sort_values(by='Date')

# Reset index for easier handling
data.reset_index(drop=True, inplace=True)


In [8]:
# Fill missing dates or values if needed
data = data.set_index('Date').resample('D').ffill().reset_index()

ValueError: cannot reindex on an axis with duplicate labels

In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

# Use 'Price' for prediction
prices = data['Price'].values

# Normalize the prices
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_prices = scaler.fit_transform(prices.reshape(-1, 1))

# Define a function to create sequences for LSTM
def create_sequences(data, seq_length):
    xs, ys = [], []
    for i in range(len(data) - seq_length):
        x = data[i:i+seq_length]
        y = data[i+seq_length]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

SEQ_LENGTH = 30
X, y = create_sequences(scaled_prices, SEQ_LENGTH)


ModuleNotFoundError: No module named 'sklearn'

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

# Split the data
split = int(len(X) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Build the model
model = Sequential()
model.add(LSTM(50, return_sequences=True, input_shape=(SEQ_LENGTH, 1)))
model.add(LSTM(50))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.1)

# Predict and evaluate
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions)
y_test = scaler.inverse_transform(y_test)

mse = mean_squared_error(y_test, predictions)
print(f"Mean Squared Error: {mse}")

# Plot the results
plt.figure(figsize=(14, 7))
plt.plot(data['Date'].values[-len(y_test):], y_test, label='Actual Prices')
plt.plot(data['Date'].values[-len(predictions):], predictions, label='Predicted Prices')
plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Price Prediction vs Actual Prices')
plt.legend()
plt.show()


RuntimeError: module compiled against API version 0xf but this version of numpy is 0xd

ImportError: initialization failed